In [2]:
from rapidfuzz import process, fuzz
import pandas as pd
import re
import numpy as np

In [3]:
#prepare loan dataset
MD_CD_raw = pd.read_excel("../../Data/Post1790/MD/MD_post1790_CD.xlsx",
                      header = 11, usecols = 'G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
MD_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
MD_CD_raw['Name'] = MD_CD_raw['First Name'].apply(lambda x: "" if pd.isnull(x) else x) + " " + MD_CD_raw['Last Name'].apply(lambda x: "" if pd.isnull(x) else x)
MD_CD_raw['Name'] = MD_CD_raw['Name'].apply(lambda x: re.sub(' +', ' ', x).strip())
MD_CD_raw['occupation1'] = MD_CD_raw['occupation1'].apply(lambda x: "" if pd.isnull(x) else x)

In [4]:
grouped_df = MD_CD_raw.groupby(['Name', 'town1', 'occupation1']).agg({'6p_Dollar': ['sum', 'count']}).reset_index()
grouped_df.columns = ['Name', 'Town', 'occupation', '6p_total', '6p_count']
grouped_df = grouped_df.sort_values('6p_total', ascending = False).reset_index(drop = True)
grouped_df['pct_cum'] = grouped_df['6p_total'].cumsum()/grouped_df['6p_total'].sum()
grouped_df['pct'] = grouped_df['6p_total']/grouped_df['6p_total'].sum()

In [5]:
MD_CD_raw[MD_CD_raw['Last Name'] == 'Harwood']

,First Name,Last Name,town1,state1,occupation1,6p_Dollar,6p_Cents,First Name.1,Last Name.1,town2,...,6p_def_Dollar,6p_def_Cents,First Name.2,Last Name.2,town3,state3,occupation3,3p_Dollar,3p_Cents,Name
152,Benjamin,Harwood,Annapolis,MD,,13700.0,94.0,Benjamin,Harwood,Annapolis,...,6850.0,46.0,Benjamin,Harwood,Annapolis,MD,NaN,7487.0,1.0,Benjamin Harwood
156,Benjamin,Harwood,Annapolis,MD,,262138.0,43.0,Benjamin,Harwood,Annapolis,...,131069.0,21.0,Benjamin,Harwood,Annapolis,MD,NaN,152239.0,70.0,Benjamin Harwood
186,Benjamin,Harwood,Annapolis,MD,,233.0,33.0,Benjamin,Harwood,Annapolis,...,116.0,67.0,Benjamin,Harwood,Annapolis,MD,NaN,221.0,56.0,Benjamin Harwood
340,Benjamin,Harwood,Annapolis,MD,,28.0,89.0,Benjamin,Harwood,Annapolis,...,14.0,44.0,Benjamin,Harwood,Annapolis,MD,NaN,18.0,49.0,Benjamin Harwood
369,Benjamin,Harwood,Annapolis,MD,,109619.0,64.0,Benjamin,Harwood,Annapolis,...,54809.0,82.0,Benjamin,Harwood,Annapolis,MD,NaN,108192.0,72.0,Benjamin Harwood
377,William,Harwood,Annapolis,MD,,118.0,70.0,William,Harwood,Annapolis,...,59.0,35.0,Thomas and Benjamin,Harwood,Annapolis,MD,NaN,145.0,98.0,William Harwood
401,Benjamin,Harwood,Annapolis,MD,,33605.0,38.0,Benjamin,Harwood,Annapolis,...,16802.0,69.0,Benjamin,Harwood,Annapolis,MD,NaN,30160.0,83.0,Benjamin Harwood
511,Benjamin,Harwood,Annapolis,MD,,666.0,67.0,Benjamin,Harwood,Annapolis,...,333.0,33.0,Benjamin,Harwood,Annapolis,MD,NaN,420.0,NaN,Benjamin Harwood


In [7]:
grouped_df.head(10)

,Name,Town,occupation,6p_total,6p_count,pct_cum,pct
0,Benjamin Harwood,Annapolis,,419989.0,7,0.458947,0.458947
1,William Marbury,Annapolis,,55986.0,39,0.520126,0.061179
2,Uriah Forrest and Benjamin Stoddert,George Town,Merchants,29873.0,2,0.552770,0.032644
3,Wallace and Muir,Annapolis,,27925.0,16,0.583286,0.030515
4,James Williams,Annapolis,,23680.0,9,0.609162,0.025877
5,John Laird,George Town,Merchant,21854.0,1,0.633044,0.023881
6,John Mason,George Town,,21307.0,2,0.656327,0.023283
7,William Campbell,Annapolis,,18761.0,9,0.676828,0.020501
8,Gustavus Scott,Cambridge,,18624.0,4,0.697180,0.020352
9,Randolph B Latimer,Annapolis,,12331.0,11,0.710655,0.013475


In [6]:
loan = pd.read_csv('../../Data/Pre1790/cleaned/loan_office_certificates_9_states_cleaned.csv', index_col = 0)
marine = pd.read_csv('../../Data/Pre1790/cleaned/Marine_Liquidated_Debt_Certificates_cleaned.csv', index_col = 0)
pierce = pd.read_csv('../../Data/Pre1790/cleaned/Pierce_Certs_cleaned_2021.csv', index_col = 0)
liquidCT = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_CT_cleaned.csv', index_col = 0)
liquidDE = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_DE_cleaned.csv', index_col = 0)
liquidMA = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_MA_cleaned.csv', index_col = 0)
liquidNJ = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NJ_cleaned.csv', index_col = 0)
liquidNH = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NH_cleaned.csv', index_col = 0)
liquidNY = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_NY_cleaned.csv', index_col = 0)
liquidPA1 = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_story_cleaned.csv', index_col = 0)
liquidRI = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_RI_cleaned.csv', index_col = 0)
liquidPA2 = pd.read_csv('../../Data/Pre1790/cleaned/liquidated_debt_certificates_PA_stelle_cleaned.csv', index_col = 0)

/var/folders/5s/dvrxt95949x1pm_sjxm85lj00000gn/T/ipykernel_72546/3372110578.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  pierce = pd.read_csv('../../Data/Pre1790/cleaned/Pierce_Certs_cleaned_2021.csv', index_col = 0)


In [7]:
def nameLiquidated(fname, lname):
    ctsum = liquidCT[[ln in lname and fn in fname for ln, fn in zip(liquidCT['Last name'],
                                                                    liquidCT['First name'])]]['Dollars'].sum()
    desum = liquidDE[[ln in lname and fn in fname for ln, fn in zip(liquidDE['Last name'],
                                                                    liquidDE['First name'])]]['Dollars'].sum()
    masum = liquidMA[[ln in lname and fn in fname for ln, fn in zip(liquidMA['Last name'],
                                                                    liquidMA['First name'])]]['Dollars'].sum()
    njsum = liquidNJ[[ln in lname and fn in fname for ln, fn in zip(liquidNJ['Last name'],
                                                                    liquidNJ['First name'])]]['Dollars'].sum()
    nhsum = liquidNH[[ln in lname and fn in fname for ln, fn in zip(liquidNH['Last name'],
                                                                    liquidNH['First name'])]]['Dollars'].sum()
    pasum1 = liquidPA1[[ln in lname and fn in fname for ln, fn in zip(liquidPA1['Last name'],
                                                                      liquidPA1['First name'])]]['Dollars'].sum()
    risum = liquidRI[[ln in lname and fn in fname for ln, fn in zip(liquidRI['Last name'],
                                                                    liquidRI['First name'])]]['Dollars'].sum()
    pasum2 = liquidPA2[[ln in lname and fn in fname for ln, fn in zip(liquidPA2['Last name'],
                                                                      liquidPA2['First name'])]]['Dollars'].sum()
    return ctsum + desum + masum + njsum + nhsum + pasum1 + risum + pasum2

## Benjamin Harwood

In [8]:
pierce[[ln == 'Harwood' and fn in ['Benjamin', 'Thomas'] for ln, fn in zip(pierce['Last'],
                                                                           pierce['First'])]]['Value'].sum()

877.8800000000001

In [9]:
marine[[ln == 'Harwood' and fn in ['Benjamin', 'Thomas'] for ln, fn in zip(marine['Last name'],
                                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [10]:
loan[[ln == 'Harwood' and fn in ['Benjamin', 'Thomas'] for ln, fn in zip(loan['Last Name 1 '],
                                                                        loan['First Name 1 '])]]['Face Value'].sum()

2400

In [11]:
nameLiquidated(['Harwood'], ['Benjamin', 'Thomas'])

0.0

## William Marbury

In [12]:
pierce[[ln == 'Marbury' and fn in ['William'] for ln, fn in zip(pierce['Last'],
                                                                pierce['First'])]]['Value'].sum()

0.0

In [13]:
marine[[ln == 'Marbury' and fn in ['William'] for ln, fn in zip(marine['Last name'],
                                                                marine['First name'])]]['Dollars'].sum()

0.0

In [14]:
loan[[ln == 'Marbury' and fn in ['William'] for ln, fn in zip(loan['Last Name 1 '],
                                                              loan['First Name 1 '])]]['Face Value'].sum()

0

In [15]:
nameLiquidated(['William'], ['Marbury'])

0.0

## Uriah Forrest and Benjamin Stoddert

In [16]:
marine[[ln in ['Stoddert', 'Forrest'] and fn in ['Uriah', 'Benjamin'] for ln, fn in zip(marine['Last name'],
                                                                                        marine['First name'])]]['Dollars'].sum()

0.0

In [17]:
pierce[[ln in ['Stoddert', 'Forrest'] and fn in ['Uriah', 'Benjamin'] for ln, fn in zip(pierce['Last'],
                                                                                        pierce['First'])]]['Value'].sum()

426.15999999999997

In [18]:
loan[[ln in ['Stoddert', 'Forrest'] and fn in ['Uriah', 'Benjamin'] for ln, fn in zip(loan['Last Name 1 '],
                                                                                      loan['First Name 1 '])]]['Face Value'].sum()

3200

In [19]:
nameLiquidated(['Uriah', 'Benjamin'], ['Stoddert', 'Forrest'])

0.0

## Wallace and Muir

In [20]:
marine[[ln == 'Muir' and fn == 'John' or ln == 'Wallace' and fn == 'Charles'  for ln, fn in zip(marine['Last name'],
                                                                                                marine['First name'])]]['Dollars'].sum()

0.0

In [21]:
pierce[[ln == 'Muir' and fn == 'John' or ln == 'Wallace' and fn == 'Charles'  for ln, fn in zip(pierce['Last'],
                                                                                                pierce['First'])]]['Value'].sum()

28.52

In [22]:
loan[[ln == 'Muir' and fn == 'John' or ln == 'Wallace' and fn == 'Charles'  for ln, fn in zip(loan['Last Name 1 '],
                                                                                              loan['First Name 1 '])]]['Face Value'].sum()

15300

In [23]:
nameLiquidated(['John', 'Charles'], ['Muir', 'Wallace'])

0.0

## James Williams

In [24]:
pierce[[ln == 'Williams' and fn in ['James'] and 'Maryland' in issue for ln, fn, issue in zip(pierce['Last'],
                                                                                              pierce['First'],
                                                                                              pierce['To Whom Issued'])]]['Value'].sum()

101.3

In [25]:
marine[[ln == 'Williams' and fn in ['James'] for ln, fn in zip(marine['Last name'],
                                                                marine['First name'])]]['Dollars'].sum()

0.0

In [26]:
loan[[ln == 'Williams' and fn in ['James'] for ln, fn in zip(loan['Last Name 1 '],
                                                             loan['First Name 1 '])]]['Face Value'].sum()

2300

In [27]:
nameLiquidated(['James'], ['Williams'])

44.0

## John Laird

In [28]:
pierce[[ln == 'Laird' and fn in ['John'] and 'Maryland' in issue for ln, fn, issue in zip(pierce['Last'],
                                                                                          pierce['First'],
                                                                                          pierce['To Whom Issued'])]]['Value'].sum()

0.0

In [29]:
marine[[ln == 'Laird' and fn in ['John'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [30]:
loan[[ln == 'Laird' and fn in ['John'] for ln, fn in zip(loan['Last Name 1 '],
                                                         loan['First Name 1 '])]]['Face Value'].sum()

0

In [31]:
nameLiquidated(['John'], ['Laird'])

0.0

## John Mason

In [58]:
pierce[[ln == 'Mason' and fn in ['John', 'Robert'] and 'Maryland' in issue for ln, fn, issue in zip(pierce['Last'],
                                                                  pierce['First'],
                                                                  pierce['To Whom Issued'])]]['Value'].sum()

0.0

In [61]:
marine[[ln == 'Mason' and fn in ['John', 'Robert'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [59]:
loan[[ln == 'Mason' and fn in ['John', 'Robert'] and state == 8 for ln, fn, state in zip(loan['Last Name 1 '],
                                                                               loan['First Name 1 '],
                                                                               loan['State'])]]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible


In [62]:
nameLiquidated(['John', 'Robert'], ['Mason'])

0.0

## William Campbell

In [36]:
pierce[[ln == 'Campbell' and fn in ['William'] for ln, fn, issue in zip(pierce['Last'],
                                                                  pierce['First'],
                                                                  pierce['To Whom Issued'])]]

,CN,Last,First,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text
12794,33475.0,Campbell,William,40.00,47.0,Regiment paid to August 1 1780,NaN,0.0,NaN,NaN,NaN
12795,36337.0,Campbell,William,26.38,54.0,Regiment paid to January 1 1783,NaN,0.0,NaN,NaN,NaN
12796,49258.0,Campbell,William,90.78,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12797,49955.0,Campbell,William,108.00,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12798,50691.0,Campbell,William,16.47,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12799,51175.0,Campbell,William,79.10,67.0,Second Artillery under Col. John Crane Massa...,MA,0.0,NaN,NaN,NaN
12800,52727.0,Campbell,William,1.70,69.0,Eighth Massachusetts Regiment under Col. Mich...,MA,0.0,NaN,NaN,NaN
12801,58673.0,Campbell,William,92.40,74.0,Invalid Corps of United States under Col. L. ...,NaN,0.0,NaN,NaN,NaN
12802,59368.0,Campbell,William,17.86,74.0,Invalid Corps of United States under Col. L. ...,NaN,0.0,NaN,NaN,NaN
12803,76747.0,Campbell,William,142.12,96.0,Pennsylvania Artillery under Lieut. Col. F. P...,PA,0.0,NaN,NaN,NaN


In [37]:
marine[[ln == 'Campbell' and fn in ['William'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [42]:
loan[[ln == 'Campbell' and fn in ['William'] and state == 8 for ln, fn, state in zip(loan['Last Name 1 '],
                                                                               loan['First Name 1 '],
                                                                               loan['State'])]]

,State,Year,Month,Day,Title 1,First Name 1,Last Name 1,Title 2,First Name 2,Last Name 2,Title 3,First Name 3,Last Name 3,Face Value,Specie Value,notes,original text,match eligible


In [41]:
nameLiquidated(['William'], ['Campbell'])

119.0

## Gustavus Scott

In [43]:
pierce[[ln == 'Scott' and fn in ['Gustavus'] for ln, fn, issue in zip(pierce['Last'],
                                                                  pierce['First'],
                                                                  pierce['To Whom Issued'])]]

,CN,Last,First,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text


In [44]:
marine[[ln == 'Scott' and fn in ['Gustavus'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [48]:
loan[[ln == 'Scott' and fn in ['Gustavus'] and state == 8 for ln, fn, state in zip(loan['Last Name 1 '],
                                                                               loan['First Name 1 '],
                                                                               loan['State'])]]['Face Value'].sum()

5200

In [47]:
nameLiquidated(['Gustavus'], ['Scott'])

0.0

## Randolph B Latimer

In [49]:
pierce[[ln == 'Latimer' and fn in ['Randolph'] for ln, fn, issue in zip(pierce['Last'],
                                                                  pierce['First'],
                                                                  pierce['To Whom Issued'])]]

,CN,Last,First,Value,Group,To Whom Issued,State,Officer,Last 2,First 2,original text


In [50]:
marine[[ln == 'Latimer' and fn in ['Randolph'] for ln, fn in zip(marine['Last name'],
                                                           marine['First name'])]]['Dollars'].sum()

0.0

In [54]:
loan[[ln == 'Latimer' and fn in ['Randolph'] and state == 8 for ln, fn, state in zip(loan['Last Name 1 '],
                                                                               loan['First Name 1 '],
                                                                               loan['State'])]]['Face Value'].sum()

0

In [55]:
nameLiquidated(['Latimer'], ['Randolph'])

0.0